In [ ]:
!pip install langchain openai gradio

In [ ]:
!pip install tiktoken

In [ ]:
!pip install faiss-gpu

In [ ]:
import os
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import FAISS
from langchain.chains.question_answering import load_qa_chain
from langchain.llms import OpenAI
from langchain.prompts import PromptTemplate
import tiktoken

In [ ]:
# Get your API keys from openai, you will need to create an account.
# Here is the link to get the keys: https://platform.openai.com/account/billing/overview
import os
from google.colab import userdata
os.environ["OPENAI_API_KEY"] = userdata.get('OpenAI')


In [ ]:
# Read the incorrect answers from the text file
with open('incorrect_answers.txt', 'r') as file:
    raw_text = file.read()
# Split the text into smaller chunks
text_splitter = CharacterTextSplitter(
    separator="\n",
    chunk_size=1000,
    chunk_overlap=200,
    length_function=len,
)

In [ ]:
texts = text_splitter.split_text(raw_text)
print(f"Number of text chunks: {len(texts)}")

Number of text chunks: 1


In [ ]:
# Set up the prompt template for identifying the main topics
topic_identification_prompt_template = """
You are an education specialist whose job is to identify the main topics that a student is lacking in based on their incorrect answers.

The context of the incorrect answers is as follows:
{context}

Based on the given context, identify the main topics that the student is struggling with. Provide a concise list of the topics.

Topic 1:
Topic 2:
Topic 3:
"""

topic_identification_prompt = PromptTemplate(template=topic_identification_prompt_template, input_variables=["context"])


In [ ]:
# Download embeddings from OpenAI
embeddings = OpenAIEmbeddings()
docsearch = FAISS.from_texts(texts, embeddings)

In [ ]:
# Set up the LLM and the chain
llm = OpenAI(temperature=0.7)
chain = load_qa_chain(llm, chain_type="stuff")

In [ ]:
# Identify the main topics the student is lacking in
topics = []
for text in texts:
    query = topic_identification_prompt.format(context=text)
    docs = docsearch.similarity_search(query)
    result = chain.run(input_documents=docs, question=query)
    topics.extend([topic.strip() for topic in result.split('\n') if topic.strip()])
topics = list(set(topics))
print("Main topics the student is lacking in:")
for topic in topics:
    print(f"- {topic}")

Main topics the student is lacking in:
- I don't know.


In [ ]:
# Set up the prompt template for creating a tailored lesson plan
lesson_plan_prompt_template = """
You are an experienced instructor who needs to create a tailored lesson plan for a student based on the main topics they are lacking in. Keep it brief.

The main topics the student is struggling with are:

{topics}

Create a comprehensive lesson plan that covers these topics. The lesson plan should include the following sections:

Lesson Objectives:

Prerequisites:

Lesson Outline:

- Introduction (5-10 minutes)
  - Overview of the lesson and its objectives
- Topic 1 (20-30 minutes)
  - Explanation of the topic
  - Examples and practice exercises
- Topic 2 (20-30 minutes)
  - Explanation of the topic
  - Examples and practice exercises
- Topic 3 (20-30 minutes)
  - Explanation of the topic
  - Examples and practice exercises
- Summary and Q&A (10-15 minutes)
  - Review of the key concepts
  - Opportunity for students to ask questions

Teaching Strategies:

Assessment and Evaluation Strategies:

Make sure the lesson plan is tailored to the specific needs of the student based on the identified main topics. More importantly, make assumptions wherever needed.
"""

In [ ]:
lesson_plan_prompt = PromptTemplate(template=lesson_plan_prompt_template, input_variables=["topics"])

In [ ]:
# Generate the tailored lesson plan
lesson_plan_query = lesson_plan_prompt.format(topics="\n- ".join(topics))
lesson_plan_docs = docsearch.similarity_search(lesson_plan_query)
lesson_plan = chain.run(input_documents=lesson_plan_docs, question=lesson_plan_query)

In [ ]:
print("\nTailored Lesson Plan:")
print(lesson_plan)


Tailored Lesson Plan:

Lesson Objectives:
- To understand the key concepts and principles of the main topics the student is struggling with
- To be able to apply these concepts in practice exercises
- To gain confidence in the topics and be able to ask questions for clarification

Prerequisites:
- Basic knowledge of the subject
- Familiarity with the terminology and key concepts

Lesson Outline:
- Introduction (5-10 minutes)
  - Brief overview of the lesson and its objectives
  - Explanation of why these topics are important and how they relate to the overall subject
- Topic 1 (20-30 minutes)
  - Explanation of the topic with examples to demonstrate its importance and relevance
  - Practice exercises to reinforce understanding
- Topic 2 (20-30 minutes)
  - Explanation of the topic with examples to demonstrate its importance and relevance
  - Practice exercises to reinforce understanding
- Topic 3 (20-30 minutes)
  - Explanation of the topic with examples to demonstrate its importance 

In [ ]:
import os
import gradio as gr

# Set up the prompt template for identifying the main topics
topic_identification_prompt_template = """
You are an education specialist whose job is to identify the main topics that a student is lacking in based on their incorrect answers.

The context of the incorrect answers is as follows:

{context}

Based on the given context, identify the main topics that the student is struggling with. Provide a concise list of the topics.

Topic 1:

Topic 2:

Topic 3:
"""

topic_identification_prompt = PromptTemplate(template=topic_identification_prompt_template, input_variables=["context"])

# Set up the prompt template for creating a tailored lesson plan
lesson_plan_prompt_template = """
You are an experienced instructor who needs to create a tailored lesson plan for a student based on the main topics they are lacking in. Keep it brief.

The main topics the student is struggling with are:

{topics}

Create a comprehensive lesson plan that covers these topics. The lesson plan should include the following sections:

Lesson Objectives:

Prerequisites:

Lesson Outline:

- Introduction (5-10 minutes)
  - Overview of the lesson and its objectives
- Topic 1 (20-30 minutes)
  - Explanation of the topic
  - Examples and practice exercises
- Topic 2 (20-30 minutes)
  - Explanation of the topic
  - Examples and practice exercises
- Topic 3 (20-30 minutes)
  - Explanation of the topic
  - Examples and practice exercises
- Summary and Q&A (10-15 minutes)
  - Review of the key concepts
  - Opportunity for students to ask questions

Teaching Strategies:

Assessment and Evaluation Strategies:

Make sure the lesson plan is tailored to the specific needs of the student based on the identified main topics. More importantly, make assumptions wherever needed.
"""

lesson_plan_prompt = PromptTemplate(template=lesson_plan_prompt_template, input_variables=["topics"])

# Set up the LLM and the chain
embeddings = OpenAIEmbeddings()
docsearch = None
llm = OpenAI(temperature=0.7)
chain = load_qa_chain(llm, chain_type="stuff")

def generate_lesson_plan(file):
    with open("incorrect_answers.txt", "w") as f:
        f.write(file)

    with open("incorrect_answers.txt", "r") as file:
        raw_text = file.read()

    # Split the text into smaller chunks
    text_splitter = CharacterTextSplitter(
        separator="\n",
        chunk_size=1000,
        chunk_overlap=200,
        length_function=len,
    )
    texts = text_splitter.split_text(raw_text)
    print(f"Number of text chunks: {len(texts)}")

    # Download embeddings from OpenAI
    global docsearch
    docsearch = FAISS.from_texts(texts, embeddings)

    # Identify the main topics the student is lacking in
    topics = []
    for text in texts:
        query = topic_identification_prompt.format(context=text)
        docs = docsearch.similarity_search(query)
        result = chain.run(input_documents=docs, question=query)
        topics.extend([topic.strip() for topic in result.split('\n') if topic.strip()])
    topics = list(set(topics))
    print("Main topics the student is lacking in:")
    for topic in topics:
        print(f"- {topic}")

    # Generate the tailored lesson plan
    lesson_plan_query = lesson_plan_prompt.format(topics="\n- ".join(topics))
    lesson_plan_docs = docsearch.similarity_search(lesson_plan_query)
    lesson_plan = chain.run(input_documents=lesson_plan_docs, question=lesson_plan_query)

    return lesson_plan

# Create the Gradio interface
with gr.Blocks() as demo:
    with gr.Row():
        with gr.Column():
            file = gr.File(label="Incorrect Answers (upload 'incorrect_answers.txt')")
            submit_btn = gr.Button("Generate Lesson Plan")
        with gr.Column():
            lesson_plan_output = gr.Textbox(label="Tailored Lesson Plan", interactive=False)

    submit_btn.click(generate_lesson_plan, inputs=file, outputs=lesson_plan_output)

demo.launch()

Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://67cd72dd213727b8af.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


In [ ]:
!pip install gradio_client

In [ ]:
from gradio_client import Client, file

client = Client("https://bb8f6c5a6f10b67bac.gradio.live/")
result = client.predict(
		file=file('https://github.com/gradio-app/gradio/raw/main/test/test_files/sample_file.pdf'),
		api_name="/generate_lesson_plan"
)
print(result)

Loaded as API: https://bb8f6c5a6f10b67bac.gradio.live/ ✔
Number of text chunks: 1
Main topics the student is lacking in:
- I don't know what topics the student is struggling with based on the given context.


Lesson Objectives:
- To understand the main topics that the student is lacking in
- To improve the student's understanding and mastery of these topics
- To provide opportunities for the student to practice and apply their knowledge

Prerequisites:
- Basic knowledge and understanding of the subject area
- Familiarity with relevant vocabulary and terminology
- Ability to follow instructions and participate in class activities

Lesson Outline:
- Introduction (5-10 minutes)
  - Briefly discuss the main topics the student is struggling with
  - Explain the importance of these topics and how they relate to the overall subject
  - Set clear learning objectives for the lesson 

- Topic 1 (20-30 minutes)
  - Explanation of the topic
    - Use visual aids, real-life examples, and analogies 